In [4]:
from dataloader import load_data

df = load_data(frac=1)

[2025-03-10 11:32:30] File exists
[2025-03-10 11:32:30] Loading data form local file system


/Users/alexanderlorenz/Documents/GitHub/ai-fc/src/dataloader.py:77: DtypeWarning: Columns (3,11,13,17,19,40,41,43,66,72,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path).sample(frac=frac, random_state=42)


[2025-03-10 11:33:05] File loaded with shape: (1069866, 116)


In [7]:
idx=[72,88,90,91,92,93,95,96,97,98,99,100,101,104,106,107,108,109,110,111,112,113,114,115]

for id in idx:
    print(f"column: {df.columns[id]},  unique values: {df[df.columns[id]].unique()}")

column: shot_one_on_one,  unique values: [nan True]
column: bad_behaviour_card,  unique values: [nan 'Yellow Card' 'Second Yellow' 'Red Card']
column: injury_stoppage_in_chain,  unique values: [nan True]
column: miscontrol_aerial_won,  unique values: [nan True]
column: pass_no_touch,  unique values: [nan True]
column: pass_straight,  unique values: [nan True]
column: foul_committed_type,  unique values: [nan 'Handball' 'Backpass Pick' 'Foul Out' 'Dangerous Play' 'Dive']
column: foul_committed_offensive,  unique values: [nan True]
column: shot_deflected,  unique values: [nan True]
column: clearance_other,  unique values: [nan True]
column: goalkeeper_success_out,  unique values: [nan True]
column: pass_miscommunication,  unique values: [nan True]
column: dribble_no_touch,  unique values: [nan True]
column: shot_open_goal,  unique values: [nan True]
column: goalkeeper_punched_out,  unique values: [nan True]
column: shot_redirect,  unique values: [nan True]
column: goalkeeper_shot_saved_o

In [6]:
df["bad_behaviour_card"].unique()

array([nan, 'Yellow Card', 'Second Yellow', 'Red Card'], dtype=object)